In [212]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
import _pickle as cPickle

In [213]:
df = pd.read_csv('data_ml.csv')

Vamos a empezar a trabajar con el modelo de ML!!

Primero separamos la variable predecir.

In [214]:
X = df.drop('price', axis=1)
y = df['price']

Instanciamos el modelo

In [215]:
regressor = LGBMRegressor()

Creamos la grilla de parámetros.

In [216]:
param_grid = {"n_estimators" : [210, 220, 230],
             "learning_rate" : [0.21, 0.22, 0.24,],
              "max_depth" : [4, 5, 6]}

- learning_rate: La tasa de aprendizaje controla cuánto se ajustan los valores de los hiperparámetros en cada paso de optimización. Un valor menor puede ayudar a mejorar la convergencia, pero también podría requerir más iteraciones.

- n_estimators: El número de árboles en el modelo. Puede aumentar la capacidad del modelo al agregar más árboles, pero también aumentará el tiempo de entrenamiento.

- max_depth: La profundidad máxima de cada árbol. Puede controlar la complejidad del modelo y prevenir el sobreajuste.

- min_child_samples: El número mínimo de muestras en un nodo hoja. Puede controlar la regularización y evitar divisiones en nodos con muy pocas muestras.

- subsample: La fracción de muestras utilizadas para entrenar cada árbol. Puede controlar la varianza y prevenir el sobreajuste.

- colsample_bytree o colsample_bylevel: La fracción de características utilizadas para entrenar cada árbol o nivel. Puede controlar la varianza y mejorar la diversidad de árboles.

- reg_alpha y reg_lambda: Parámetros de regularización L1 y L2, respectivamente. Pueden ayudar a evitar el sobreajuste y controlar la complejidad del modelo.

- min_split_gain: Ganancia mínima requerida para realizar una división en un nodo. Puede controlar la cantidad de mejora requerida para agregar un nodo adicional.

- num_leaves: Número máximo de hojas en cada árbol. Puede controlar la complejidad del modelo y la capacidad de adaptación a los datos.

- scale_pos_weight: Utilizado en problemas de clasificación desequilibrados para ajustar el peso de las clases positivas y negativas.

Instanciamos el método GridSearchCV.

In [217]:
grid_search = GridSearchCV(regressor, param_grid, scoring='neg_mean_squared_error', cv=5)

Fiteamos el modelo.

In [218]:
grid_search.fit(X, y)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009417 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 692
[LightGBM] [Info] Number of data points in the train set: 20624, number of used features: 321
[LightGBM] [Info] Start training from score 8.896889
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

GridSearchCV(cv=5, estimator=LGBMRegressor(),
             param_grid={'learning_rate': [0.21, 0.22, 0.24],
                         'max_depth': [4, 5, 6],
                         'n_estimators': [210, 220, 230]},
             scoring='neg_mean_squared_error')

In [219]:
best_params = grid_search.best_params_
best_params

{'learning_rate': 0.22, 'max_depth': 5, 'n_estimators': 220}

In [220]:
best_mse = grid_search.best_score_
rmse = np.sqrt(-best_mse)

In [221]:
rmse

8.14425730002838

Entrenamos el modelo con los mejores hiperparametros.

In [222]:
lgbmr = LGBMRegressor(learning_rate=0.2, max_depth=6, n_estimators=220)

In [223]:
lgbmr.fit(X, y)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021838 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 714
[LightGBM] [Info] Number of data points in the train set: 25781, number of used features: 332
[LightGBM] [Info] Start training from score 8.954915
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fu

LGBMRegressor(learning_rate=0.2, max_depth=6, n_estimators=220)

Guardamos el modelo como archivo pickle.

<p><img src="src/03_pickle.png" height=200> <p>

In [224]:
with open('lgbm_regressor_model.pkl', 'wb') as model:
    cPickle.dump(lgbmr, model)

Probamos levantar el modelo de LGBMRegression, y lo llamamos con otro nombre

In [225]:
with open('lgbm_regressor_model.pkl', 'rb') as modelo:
        modelo_lgbm = cPickle.load(modelo)

Podemos ver la cantidad de features usadas por este modelo, ya que lo guardamos ya entrenado.

In [226]:
print(modelo_lgbm.n_features_)

380


In [227]:
prediccion = modelo_lgbm.predict(X.sample(1, random_state=17))

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [228]:
f"Precio predicho: {prediccion[0]}"

'Precio predicho: 4.21250997285766'

In [229]:
rmse = np.sqrt(mean_squared_error(y.sample(1, random_state=17), prediccion))

print("RMSE:", rmse)

RMSE: 1.2225099728576598


Vamos a preparar un df con algunos valores seteados en 0, para luego pasarle los datos para hacer la predicción.

In [230]:
x_prediccion = X.head(1)
lista_columnas = X.columns.to_list()
x_prediccion

,early_access,sentiment,publisher_cat,developer_cat,year,month_2,month_3,month_4,month_5,month_6,...,Warhammer 40K,Web Publishing,Werewolves,Western,Word Game,World War I,World War II,Wrestling,Zombies,e-sports
0,False,0.0,1,1,2018,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0


In [231]:
lista_columnas[:17]

['early_access',
 'sentiment',
 'publisher_cat',
 'developer_cat',
 'year',
 'month_2',
 'month_3',
 'month_4',
 'month_5',
 'month_6',
 'month_7',
 'month_8',
 'month_9',
 'month_10',
 'month_11',
 'month_12',
 "1990's"]

In [232]:
lista_features = lista_columnas[16:]
lista_features[0]

"1990's"

In [233]:
x_prediccion.loc[:,lista_features] = 0
x_prediccion

,early_access,sentiment,publisher_cat,developer_cat,year,month_2,month_3,month_4,month_5,month_6,...,Warhammer 40K,Web Publishing,Werewolves,Western,Word Game,World War I,World War II,Wrestling,Zombies,e-sports
0,False,0.0,1,1,2018,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0


In [234]:
x_prediccion.columns = [column.lower() for column in x_prediccion.columns]
x_prediccion

,early_access,sentiment,publisher_cat,developer_cat,year,month_2,month_3,month_4,month_5,month_6,...,warhammer 40k,web publishing,werewolves,western,word game,world war i,world war ii,wrestling,zombies,e-sports
0,False,0.0,1,1,2018,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0


In [235]:
with open('x_prediccion.pkl', 'wb') as x_pred:
    cPickle.dump(x_prediccion, x_pred)

In [236]:
with open('x_prediccion.pkl', 'rb') as x_prediccion:
    x_pred = cPickle.load(x_prediccion)

x_pred.columns

Index(['early_access', 'sentiment', 'publisher_cat', 'developer_cat', 'year',
       'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       ...
       'warhammer 40k', 'web publishing', 'werewolves', 'western', 'word game',
       'world war i', 'world war ii', 'wrestling', 'zombies', 'e-sports'],
      dtype='object', length=380)

In [237]:
x_pred['sentiment'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1 entries, 0 to 0
Series name: sentiment
Non-Null Count  Dtype  
--------------  -----  
1 non-null      float64
dtypes: float64(1)
memory usage: 140.0 bytes
